In [1]:
import pandas as pd
import re
import gensim
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel, TfidfModel, Word2Vec, LsiModel, LdaMulticore, HdpModel
import numpy as np
import tqdm
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from pprint import pprint

In [2]:
raw_sop = pd.read_csv('./data/sops.csv')

In [3]:
raw_sop.head()

,city,sop_doc
0,Austin,Austin Police DepartmentPolicy ManualCopyright...
1,Austin,Austin Police DepartmentPolicy ManualCopyright...
2,Austin,Austin Police DepartmentPolicy ManualCopyright...
3,Austin,Austin Police DepartmentPolicy ManualCanons of...
4,Austin,Austin Police DepartmentPolicy ManualCanons of...


In [4]:
sop = raw_sop.drop(columns=['city'])

In [5]:
sop.sample(10)

,sop_doc
2440,Evanston Police Department\nPolicy Manual\nTra...
5117,Professional Conduct and Ethics\n of Police t...
5341,Line/Operations Procedures\n in cases in whic...
6288,Las Vegas Metropolitan Polic...
1754,-Emergency R\nesponder Highway & Interstate Sa...
6269,Las Vegas Metropolitan Polic...
2396,Evanston Police Department\nPolicy Manual\nFoo...
5398,Line/Operation\ns Procedures\n Exampl\ne: Of...
3488,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
6002,Las Vegas Metropolitan Police Department Partn...


In [6]:
sop['sop_doc_processed'] = sop[
    'sop_doc'
].map(lambda x: re.sub('[,\.!?]', '', str(x))).map(lambda x: re.sub('\n', ' ', str(x)))

<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<>:3: DeprecationWarning: invalid escape sequence \.
<ipython-input-6-da76bbe04fda>:3: DeprecationWarning: invalid escape sequence \.
  ].map(lambda x: re.sub('[,\.!?]', '', str(x))).map(lambda x: re.sub('\n', ' ', str(x)))


In [7]:
sop['sop_doc_processed'] = sop['sop_doc_processed'].map(lambda x: x.lower().strip())

In [8]:
sop = sop[sop['sop_doc_processed'] != '']

In [9]:
sop.sample(10)

,sop_doc,sop_doc_processed
160,Austin Police DepartmentPolicy ManualHandling ...,austin police departmentpolicy manualhandling ...
40,Policy118Austin Police DepartmentPolicy Manual...,policy118austin police departmentpolicy manual...
1910,DURHAM POLICE DEPARTMENT\n BLANK MEDIUM CHECK\...,durham police department blank medium check -...
693,Austin Police DepartmentPolicy ManualLimited D...,austin police departmentpolicy manuallimited d...
6268,Las Vegas Metropolitan Polic...,las vegas metropolitan police departmentpartne...
2585,Evanston Police Department\nPolicy Manual\nRec...,evanston police department policy manual recor...
64,Austin Police DepartmentPolicy ManualControl D...,austin police departmentpolicy manualcontrol d...
6106,Las Vegas Metropolitan Police DepartmentPartne...,las vegas metropolitan police departmentpartne...
475,Austin Police DepartmentPolicy ManualEquipment...,austin police departmentpolicy manualequipment...
2275,Evanston Police Department\nPolicy Manual\nDif...,evanston police department policy manual diffe...


In [10]:
sop.shape

(4883, 2)

In [11]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [12]:
data = sop.sop_doc_processed.values.tolist()
data_words = np.array(list(sent_to_words(data)))

In [13]:
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=1) # higher threshold fewer phrases.

In [14]:
trigram = gensim.models.Phrases(bigram[data_words], threshold=1)

In [15]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [16]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [17]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [18]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([
            token.lemma_ for token in doc if token.pos_ in allowed_postags
        ])
    return texts_out

In [19]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [20]:
print(data_lemmatized[:1])

[['right', 'permission', 'departmentchief', 'message', 'chief', 'proud', 'present', 'new', 'edition', 'manual', 'design', 'assist', 'employee', 'accomplish', 'professional', 'adherence', 'policy', 'also', 'help', 'safeguard', 'employee', 'ensure', 'employee', 'protect', 'individual', 'action', 'scrutinize', 'especially', 'critical', 'incidentall', 'employee', 'abide', 'policy', 'responsible', 'keep', 'current', 'content', 'safebrianbrian', 'manleychief', 'police']]


In [21]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Tfidf corpus
tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [23]:
models = {}

In [24]:
hdp_model_bow = HdpModel(
    corpus=corpus,
    id2word=id2word
)

models['HDP BOW'] = hdp_model_bow

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


In [25]:
hdp_model_tfidf = HdpModel(
    corpus=corpus_tfidf,
    id2word=id2word
)

models['HDP TFIDF'] = hdp_model_tfidf

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


In [26]:
pprint(hdp_model_bow.print_topics())

[(0,
  '0.013*officer + 0.008*department + 0.007*employee + 0.006*report + '
  '0.006*may + 0.006*person + 0.006*member + 0.005*shall + 0.005*request + '
  '0.005*police'),
 (1,
  '0.013*officer + 0.007*report + 0.007*department + 0.006*employee + '
  '0.006*member + 0.006*person + 0.006*may + 0.005*police + 0.005*request + '
  '0.005*shall'),
 (2,
  '0.010*department + 0.009*officer + 0.007*employee + 0.007*order + 0.006*may '
  '+ 0.005*report + 0.005*person + 0.005*shall + 0.005*police + 0.005*member'),
 (3,
  '0.007*officer + 0.007*employee + 0.005*supervisor + 0.004*incident + '
  '0.003*person + 0.003*shall + 0.003*duty + 0.003*may + 0.003*report + '
  '0.003*department'),
 (4,
  '0.006*officer + 0.005*employee + 0.003*may + 0.003*right + 0.002*department '
  '+ 0.002*follow + 0.002*shall + 0.002*permission + 0.002*supervisor + '
  '0.002*policy'),
 (5,
  '0.003*officer + 0.003*report + 0.003*employee + 0.003*duty + 0.003*chief + '
  '0.002*wear + 0.002*police + 0.002*supervisor 

In [27]:
pprint(hdp_model_tfidf.print_topics())

[(0,
  '0.002*report + 0.002*officer + 0.001*vehicle + 0.001*employee + '
  '0.001*member + 0.001*suspect + 0.001*person + 0.001*shall + 0.001*victim + '
  '0.001*department'),
 (1,
  '0.001*employee + 0.001*victim + 0.001*police + 0.001*member + 0.001*report '
  '+ 0.000*department + 0.000*request + 0.000*person + 0.000*service + '
  '0.000*publish'),
 (2,
  '0.000*member + 0.000*vehicle + 0.000*employee + 0.000*officer + '
  '0.000*report + 0.000*star_pointing + 0.000*person + 0.000*police + '
  '0.000*department + 0.000*apply'),
 (3,
  '0.000*item + 0.000*suspect + 0.000*arrest + 0.000*request + 0.000*unit + '
  '0.000*report + 0.000*member + 0.000*pr_omptly + 0.000*training + '
  '0.000*order'),
 (4,
  '0.000*vehicle + 0.000*information + 0.000*record + 0.000*officer + '
  '0.000*report + 0.000*survivor + 0.000*maintain + 0.000*suicide_section + '
  '0.000*order + 0.000*hoursor'),
 (5,
  '0.000*provide + 0.000*unit + 0.000*officer + 0.000*overnight_delivery + '
  '0.000*pursuit + 0

In [28]:
coherence_model_lda_bow = CoherenceModel(model=hdp_model_bow, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [29]:
coherence_model_lda_bow.get_coherence()

0.4658920048894458

In [30]:
coherence_model_lda_tfidf = CoherenceModel(model=hdp_model_tfidf, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [31]:
coherence_model_lda_tfidf.get_coherence()

0.6952127451971254

In [32]:
hdp_model_tfidf.get_topics().shape

(150, 42785)

In [33]:
lda_model = LdaMulticore(corpus_tfidf, num_topics=30, id2word=id2word, workers=3)

models['IDA 30'] = lda_model

In [34]:
pprint(lda_model.print_topics())

[(6,
  '0.001*"child" + 0.001*"employee" + 0.001*"item" + 0.001*"police" + '
  '0.001*"report" + 0.001*"member" + 0.001*"request" + 0.001*"supervisor" + '
  '0.001*"vehicle" + 0.001*"officer"'),
 (26,
  '0.001*"employee" + 0.001*"member" + 0.001*"individual" + 0.001*"officer" + '
  '0.001*"person" + 0.001*"suspect" + 0.001*"information" + 0.001*"report" + '
  '0.001*"police" + 0.001*"department"'),
 (15,
  '0.002*"vehicle" + 0.001*"officer" + 0.001*"report" + 0.001*"employee" + '
  '0.001*"shall" + 0.001*"member" + 0.001*"department" + 0.001*"incident" + '
  '0.001*"duty" + 0.001*"police"'),
 (23,
  '0.001*"employee" + 0.001*"person" + 0.001*"officer" + 0.001*"record" + '
  '0.001*"information" + 0.001*"pursuit" + 0.001*"vehicle" + 0.001*"unit" + '
  '0.001*"release" + 0.001*"member"'),
 (13,
  '0.001*"employee" + 0.001*"report" + 0.001*"supervisor" + 0.001*"officer" + '
  '0.001*"chief" + 0.001*"call" + 0.001*"request" + 0.001*"unit" + '
  '0.001*"ensure" + 0.001*"section"'),
 (14,
  

In [35]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [36]:
coherence_model_lda.get_coherence()

0.27011909973414827

In [37]:
lda_hdp_sug = hdp_model_tfidf.suggested_lda_model()

models['HDP TFIDF IDA'] = lda_model

In [38]:
lda_hdp_sug.get_topics().shape

(150, 42785)

In [39]:
coherence_model_lda_hdp_sug = CoherenceModel(
    model=lda_hdp_sug, 
    texts=data_lemmatized, 
    dictionary=id2word, 
    coherence='c_v'
)

In [40]:
coherence_model_lda_hdp_sug.get_coherence()

0.7169767396462244

In [41]:
hdp_model_tfidf = gensim.models.HdpModel(
    corpus=corpus_tfidf,
    id2word=id2word,
    alpha=10
)

/opt/anaconda3/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


In [42]:
hdp_model_tfidf.get_topics().shape

(150, 42785)

In [43]:
pprint(hdp_model_tfidf.print_topics())

[(0,
  '0.001*employee + 0.001*member + 0.001*report + 0.001*officer + '
  '0.001*vehicle + 0.001*victim + 0.001*person + 0.001*suspect + 0.001*duty + '
  '0.001*supervisor'),
 (1,
  '0.001*officer + 0.001*victim + 0.001*order + 0.001*report + 0.001*member + '
  '0.001*department + 0.001*shall + 0.001*employee + 0.001*violation + '
  '0.001*person'),
 (2,
  '0.001*report + 0.001*member + 0.001*vehicle + 0.001*police + 0.001*request '
  '+ 0.001*unit + 0.001*officer + 0.001*employee + 0.001*chief + '
  '0.001*department'),
 (3,
  '0.000*employee + 0.000*łdoe + 0.000*property + 0.000*training + '
  '0.000*vehicle + 0.000*complete + 0.000*unit + 0.000*duty + 0.000*crime + '
  '0.000*ticket'),
 (4,
  '0.000*department_general + 0.000*revise + 0.000*order + 0.000*employee + '
  '0.000*duty + 0.000*arrest + 0.000*location + 0.000*vehicle + 0.000*juvenile '
  '+ 0.000*child'),
 (5,
  '0.001*employee + 0.001*department_general + 0.000*revise + 0.000*order + '
  '0.000*member + 0.000*subsequent

In [44]:
lsi_model = LsiModel(corpus_tfidf, num_topics=150, id2word=id2word)

models['LSI'] = lsi_model

In [45]:
coherence_model_lsi = CoherenceModel(model=lsi_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [46]:
coherence_model_lsi.get_coherence()

0.31233778794607675

In [47]:
pprint(lsi_model.print_topics())

[(0,
  '0.167*"employee" + 0.142*"officer" + 0.136*"vehicle" + 0.127*"report" + '
  '0.125*"member" + 0.116*"chief" + 0.112*"shall" + 0.107*"supervisor" + '
  '0.104*"department" + 0.102*"person"'),
 (1,
  '-0.732*"department_general" + -0.383*"revise" + -0.326*"order" + '
  '-0.266*"police_revise" + -0.228*"david_brown" + -0.196*"chief" + '
  '-0.043*"safety_revise" + -0.039*"family_support" + -0.039*"team_revise" + '
  '-0.036*"personnel_table"'),
 (2,
  '0.343*"employee" + -0.268*"vehicle" + 0.230*"chief" + -0.179*"victim" + '
  '-0.170*"arrest" + -0.166*"suspect" + -0.148*"search" + 0.123*"member" + '
  '-0.120*"pursuit" + -0.119*"child"'),
 (3,
  '0.643*"vehicle" + 0.248*"pursuit" + -0.239*"victim" + -0.159*"child" + '
  '0.126*"employee" + -0.126*"investigation" + 0.116*"tow" + 0.102*"wear" + '
  '-0.101*"arrest" + -0.084*"juvenile"'),
 (4,
  '0.529*"police_revise" + -0.529*"revise" + 0.397*"david_brown" + '
  '0.330*"chief" + -0.168*"employee" + 0.124*"pursuit" + 0.120*"vehicle"

In [51]:
for key, value in models.items():
    print(key + ': ')
    pprint(value.print_topics(num_topics=5, num_words=5))
    print()
    print()

HDP BOW: 
[(0,
  '0.013*officer + 0.008*department + 0.007*employee + 0.006*report + '
  '0.006*may'),
 (1,
  '0.013*officer + 0.007*report + 0.007*department + 0.006*employee + '
  '0.006*member'),
 (2,
  '0.010*department + 0.009*officer + 0.007*employee + 0.007*order + '
  '0.006*may'),
 (3,
  '0.007*officer + 0.007*employee + 0.005*supervisor + 0.004*incident + '
  '0.003*person'),
 (4,
  '0.006*officer + 0.005*employee + 0.003*may + 0.003*right + '
  '0.002*department')]


HDP TFIDF: 
[(0,
  '0.002*report + 0.002*officer + 0.001*vehicle + 0.001*employee + '
  '0.001*member'),
 (1,
  '0.001*employee + 0.001*victim + 0.001*police + 0.001*member + 0.001*report'),
 (2,
  '0.000*member + 0.000*vehicle + 0.000*employee + 0.000*officer + '
  '0.000*report'),
 (3, '0.000*item + 0.000*suspect + 0.000*arrest + 0.000*request + 0.000*unit'),
 (4,
  '0.000*vehicle + 0.000*information + 0.000*record + 0.000*officer + '
  '0.000*report')]


IDA 30: 
[(26,
  '0.001*"employee" + 0.001*"member" + 0